# Batch processing strategies: vertical and horizontal integration

Congratulations, you've just completed an awesome image processing pipeline that takes an image and does something rather useful. You've tested it a few times and you're happy with the results. Now you need to apply the same operation to 1000 images. Maybe you need to apply them to 20 datasets with slightly different parameters. Maybe you need to aggreagate information across the results of each processed image to make your conclusions. Doing this can be hard, but sometimes it's so easy you can see many ways to approach this, and are not sure which one is the best. Let's talk about what you need to keep in mind when building systems to actually apply your pipeline.

In [1]:
%matplotlib inline

**Exercise** You're at the point where you have the functions below. How might you apply the pipeline you developed to every image in a folder, saving the results in a list?

In [2]:
from skimage.io import imread

def loading(image_file_name):
    return imread(image_file_name)

def preprocessing(image):
    return image # real code would go here

def info_extraction(image):
    results = None
    return results

data_folder = "../data"

In [ ]:
from os.path import listdir

files = listdir(data_folder)
results = []

for f in files:
    img = loading(f)
    processed = preprocessing(img)
    result = info_extraction(processed)
    results.append(result)

What made this a pretty straightforward proceedure is that we encapsulated _everything_ in a function. If we developed our pipeline without doing this, we could wrap everthing in a giant function which took as parameters everything we needed.

In [ ]:
def whole_pipeline(image_file_name):
    pass # code would go here

This may not be the best idea. Maybe next week you want to load similar images the same way, preprocess them differently, then extract the same information. You could have imported the `loading` and `info_extraction` you already have.

You may also be thinking of an alternative way of solving this problem that looks like this.

In [ ]:
files = listdir(data_folder)
images = []
processed = []
results = []

for f in files:
    images.append(loading(f))
    
for img in images:
    processed.append(preprocessing(img))
    
for proc in processed:
    results.append(info_extraction(proc))

Instead of applying all processing steps to each image one at a time, you can apply each processing step to each image. So now rather than completing your pipline "vertically", from start to finish, top to bottom, you are completing your pipeline "horizontally", applying each stage across the board in sequence.

In fact, when you've made your code modular with little functions, Python tries to support this approach with _functional programming tools_. We'll talk about one of the especially useful ones:

- `map(function, iterable)`. Applies `function` to each element in `iterable` (i.e. anything you can loop through) and gives you a new list with the results.

In [ ]:
files = listdir(data_folder)
images = map(loading, files)
processed = map(preprocessing(images))
results = map(info_extraction, processed)

Now this looks like a pretty concise way to process a pipeline! But is it the best way?

One thing to consider is that at the end of the program, `files`, `images`, `processed`, and `results` are all full arrays available to you in program memmory. This could be useful for debugging (did everything look good after preprocessing?), but may use a lot of memory.

**Exercise** Not all lines above are as memory intensive as the others. Which line is the least offensive?

**Exercise** in our lesson on loading an image we discussed how much memory an image can use up. If you are processing 1000 16-bit, 1024x1024 pixel images each with 3 channels, how much memory does the first line above use?

In [5]:
num_bytes = 1000 * 2 * 1024**2 * 3
print("{} Bytes".format(num_bytes))
print("{} GB".format(num_bytes / 1024.**3))

That's probably not going to go smoothly if you have a multi-stage pipeline. You could delete each stage as you go, keeping in memory only the last and current stage using `del` stagements, e.g. `del images`, but this can introduce bugs and still requires you to keep copies of everything. For very large numbers of images, this strategy suffers.

Memory is just one resource that you have to be conscious of while designing pipelines. Speed (or time) is a concern too. You may think you are willing to wait on code for a day or two, but almost without fail your code will crash midway through and you will need to continuously iterate on your pipeline. You don't want to wait hours to verify that your list update worked.

If you've been listening to Intel's marketing, you may be thinking "My computer has 8 cores. Can't I process 8 images at a time?"

Yes. Yes you can. Python has many tools to run code in parallel, but these tools all work by, secretly, copying your code and running another Python process on your computer. That's OK, it will manage creating, communicating with, and destroying that process, but things you may expect to work, like accessing variables you declared earlier in your code, may go poorly. However, if you've built a pipeline like we have, carefully encapsulating everything (even imports) in functions, you will be fine. Functional programming and parallel programming go hand-in-hand.

To give an example of how to write programs that run in parallel, we will give a quick example using `joblib`. Other libraries exist with different tradeoffs and limitations. `ipyparallel`, `pyspark`, and `multiprocessing` are all libraries to keep in mind.

In [6]:
def process_inputs(x):
    # because this will run in parallel, we can only assume that inputs to the function will be available
    # this includes things we imported elsewhere - we lose them when we go parallel
    return x**2

Using tools you already know, we might apply this function to some inputs like so.

In [11]:
import numpy as np
import matplotlib.pyplot as plt

In [14]:
x_values = np.random.randn(50)
y_values = map(process_inputs, x_values)
plt.plot(x_values, y_values, '.')

Let's do this with joblib

In [15]:
from joblib import Parallel

In [ ]:
Parallel.